In [1]:
import tensorflow as tf
import numpy as np
from itertools import count

from sketch_util  import batch_gather
from SketchModels import VRNN
from SketchBLSTM  import BLSTM

BATCH_SIZE = 500

# Read data from sketches in numpy file

In [2]:
# Requires first running sketch_knn_parse.ipynb

features_dir = '../features'
names = ['keys', 'lengths', 'sketches', 'labels']

def get_data(name, mode='tr'):
    assert name in names
    assert mode in ['tr', 'ev']
    
    return np.load('{}/{}_{}.npy'.format(features_dir, mode, name))

In [3]:
tr_lengths, tr_sketches = get_data('lengths', mode='tr'), get_data('sketches', mode='tr')
ev_lengths, ev_sketches = get_data('lengths', mode='ev'), get_data('sketches', mode='ev')

tr_data = tf.data.Dataset.from_tensor_slices((tr_lengths, tr_sketches)).batch(BATCH_SIZE).prefetch(2)
ev_data = tf.data.Dataset.from_tensor_slices((ev_lengths, ev_sketches)).batch(BATCH_SIZE).prefetch(2)
tr_data = tr_data.make_one_shot_iterator().get_next()
ev_data = ev_data.make_one_shot_iterator().get_next()
tr_data, ev_data

((<tf.Tensor 'IteratorGetNext:0' shape=(?,) dtype=uint16>,
  <tf.Tensor 'IteratorGetNext:1' shape=(?, 300, 3) dtype=uint8>),
 (<tf.Tensor 'IteratorGetNext_1:0' shape=(?,) dtype=uint16>,
  <tf.Tensor 'IteratorGetNext_1:1' shape=(?, 300, 3) dtype=uint8>))

# Store VRNN logits

In [4]:
vrnn = VRNN(my_flags={'batch_size': 64})
def get_vrnn_logits(data):
    lengths = tf.cast(data[0], tf.int32)
    sketch_prepr = vrnn._preprocess(sketches=data[1], lengths=lengths)
    out_dict = vrnn.model_fn({'sketch_prepr': sketch_prepr, 'length': lengths}, mode='predict')
    return batch_gather(out_dict[0]['logits_hz'], lengths)

GPU Device in use: ''
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.


From /usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/sparse_ops.py:1165: sparse_to_dense (from tensorflow.python.ops.sparse_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.


In [ ]:
tr_feats, ev_feats = [], []

tr_logits_op = get_vrnn_logits(tr_data)
ev_logits_op = get_vrnn_logits(ev_data)

print(tr_logits_op, ev_logits_op)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    vrnn.restore_latest(sess)
    for i in count():
        if (i+1)%20 == 0: print(i+1)
        try:
            tr_feats.append(sess.run(tr_logits_op))
        except tf.errors.OutOfRangeError:
            break
            
    for i in count():
        if (i+1)%20 == 0: print(i+1)
        try:
            ev_feats.append(sess.run(ev_logits_op))
        except tf.errors.OutOfRangeError:
            break

Tensor("GatherNd:0", shape=(?, 1024), dtype=float32) Tensor("GatherNd_1:0", shape=(?, 1024), dtype=float32)
INFO:tensorflow:Restoring parameters from ../checkpts/VRNNv9_hs2x512_bs64_anneal-lr_maxlen150_kldw1.0_trainTogether_annealKL-from0.4-end1.0-steps10000_cln-hidden+latent/checkpoint-79999


Restoring parameters from ../checkpts/VRNNv9_hs2x512_bs64_anneal-lr_maxlen150_kldw1.0_trainTogether_annealKL-from0.4-end1.0-steps10000_cln-hidden+latent/checkpoint-79999


Values were restored (scope: None)
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980
1000
1020


In [15]:
tr = np.concatenate(tr_feats).astype(np.float32)
ev = np.concatenate(ev_feats).astype(np.float32)
tr.shape, ev.shape

((1000000, 1024), (1700000, 1024))

In [ ]:
np.save(features_dir + '/tr_logits_vrnn.npy', tr)
np.save(features_dir + '/ev_logits_vrnn.npy', ev)

# Store BLSTM logits

In [ ]:
blstm = BLSTM(my_flags={'batch_size': 64})
def get_blstm_logits(data):
    lengths = tf.cast(data[0], tf.int32)
    sketch_prepr = blstm._preprocess(sketches=data[1], lengths=lengths)
    out_dict = blstm.model_fn({'sketch_prepr': sketch_prepr, 'length': lengths}, mode='predict')
    return out_dict[0]['logits_for_cl']

In [5]:
tr_feats, ev_feats = [], []

tr_logits_op = get_blstm_logits(tr_data)
ev_logits_op = get_blstm_logits(ev_data)

print(tr_logits_op, ev_logits_op)

config = tf.ConfigProto(
        device_count = {'GPU': 0}
    )
with tf.Session() as sess: # config=config) as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    blstm.restore_latest(sess)
    for i in count():
        if (i+1)%20 == 0: print(i+1)
        try:
            tr_feats.append(sess.run(tr_logits_op))
        except tf.errors.OutOfRangeError:
            break
            
    for i in count():
        if (i+1)%20 == 0: print(i+1)
        try:
            ev_feats.append(sess.run(ev_logits_op))
        except tf.errors.OutOfRangeError:
            break

Tensor("logits_3/GatherNd:0", shape=(?, 1024), dtype=float32) Tensor("logits_4/GatherNd:0", shape=(?, 1024), dtype=float32)
INFO:tensorflow:Restoring parameters from ../checkpts/BLSTM_hs2x512_anneal-lr_maxlen150/checkpoint-97999


Restoring parameters from ../checkpts/BLSTM_hs2x512_anneal-lr_maxlen150/checkpoint-97999


Values were restored (scope: None)
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980
1000
1020
1040
1060
1080
1100
1120
1140
1160
1180
1200
1220
1240
1260
1280
1300
1320
1340
1360
1380
1400
1420
1440
1460
1480
1500
1520
1540
1560
1580
1600
1620
1640
1660
1680
1700
1720
1740
1760
1780
1800
1820
1840
1860
1880
1900
1920
1940
1960
1980
2000
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980
1000
1020
1040
1060
1080
1100
1120
1140
1160
1180
1200
1220
1240
1260
1280
1300
1320
1340
1360
1380
1400
1420
1440
1460
1480
1500
1520
1540
1560
1580
1600
1620
1640
1660
1680
1700
1720
1740
1760
1780
1800
1820
1840
1860
1880
1900
1920
1940
1960
1980
2000
2020
2040
2060
2080
2100
2120
2140
2160
2180
2200
2220
2240
2260
2280
2

In [6]:
tr = np.concatenate(tr_feats).astype(np.float32)
ev = np.concatenate(ev_feats).astype(np.float32)
tr.shape, ev.shape

((1000000, 1024), (1700000, 1024))

In [7]:
np.save(features_dir + '/tr_logits_blstm.npy', tr)
np.save(features_dir + '/ev_logits_blstm.npy', ev)

# Store ResNetv2 Logits

In [19]:
### 
# CNN part
###
from sketch_util import create_snapshot_imgs
from tensorflow.contrib.slim      import conv2d
from tensorflow.contrib.slim.nets import resnet_v2
import os

In [12]:
features_dir = '../features'

def get_images(mode='tr'):
    def _img_transform(length, sketch):
        img_fn  = lambda length, sketch: create_snapshot_imgs(length, sketch, num_snapshots=1)
        output = tf.py_func(img_fn, [length, sketch], [tf.int32, tf.uint8, tf.int32], stateful=False)
        num_imgs    = output[0]
        imgs        = tf.reshape(output[1], (1,224*224))
        img_indices = tf.reshape(output[2], (-1,))
        return imgs[0,:]

    assert mode in ['tr', 'ev']
    
    length = np.load('{}/{}_lengths.npy'.format(features_dir, mode))
    sketch = np.load('{}/{}_sketches.npy'.format(features_dir, mode))
    
    data = tf.data.Dataset.from_tensor_slices((length, sketch))
    data = data.map(_img_transform, num_parallel_calls=4).batch(BATCH_SIZE).prefetch(2)
    return data.make_one_shot_iterator().get_next()

tr_imgs = get_images('tr')
ev_imgs = get_images('ev')

In [20]:
def get_cnn_logits(last_imgs):
    last_imgs = tf.reshape(last_imgs, [-1, 224,224,1])
    last_imgs = tf.cast(last_imgs, tf.float32)
    # CNN
    with tf.variable_scope('cnn/resnet', reuse=tf.AUTO_REUSE):
        logits, _ = resnet_v2.resnet_v2_50(last_imgs, is_training=True) # ?x1x1x2048
        logits = conv2d(inputs=logits, num_outputs=cnn_size, kernel_size=[1, 1], activation_fn=None,
                        normalizer_fn=None, scope='logits')
    logits = tf.squeeze(logits, [1,2])
    return logits

In [21]:
def restore_latest(sess, model_name, checkpoint_dir, scope=None, logging_fn=print):
        var_list = tf.global_variables(scope=scope)
        saver = tf.train.Saver(var_list=var_list)
        ckpt = tf.train.get_checkpoint_state(os.path.dirname(checkpoint_dir + '/checkpoint'))
        if ckpt and ckpt.model_checkpoint_path: # pylint: disable=E1101
            try:
                saver.restore(sess, ckpt.model_checkpoint_path) # pylint: disable=E1101
                logging_fn("Values were restored (scope: {})".format(scope))
            except:
                logging_fn("Failure restoring variables (scope: {})".format(scope))
                logging_fn("Variables in checkpoint:")
                for v in get_ckpt_vars(checkpoint_dir): print(*v)
                raise
            return True

        logging_fn("No values were restored for model {}\n\
                    Checkpoint directory: {} \n\
                    Starting with new weights".format(model_name, checkpoint_dir))
        return False

In [22]:
batch_size = 64
cnn_size = 256
name = 'conv'

model_name = name + '_bs' + str(batch_size) + '_cnnSize' + str(cnn_size)
checkpoint_dir  = '../checkpts/'  + model_name + '/'

tr_feats, ev_feats = [], []

tr_logits_op = get_cnn_logits(tr_imgs)
ev_logits_op = get_cnn_logits(ev_imgs)

with tf.Session() as sess: # config=config) as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())

    restore_latest(sess, model_name, checkpoint_dir)
    for i in count():
        if (i+1)%20 == 0: print(i+1)
        try:
            tr_feats.append(sess.run(tr_logits_op))
        except tf.errors.OutOfRangeError:
            break
            
    for i in count():
        if (i+1)%20 == 0: print(i+1)
        try:
            ev_feats.append(sess.run(ev_logits_op))
        except tf.errors.OutOfRangeError:
            break


INFO:tensorflow:Restoring parameters from ../checkpts/conv_bs64_cnnSize256/checkpoint-39600
Values were restored (scope: None)
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980
1000
1020
1040
1060
1080
1100
1120
1140
1160
1180
1200
1220
1240
1260
1280
1300
1320
1340
1360
1380
1400
1420
1440
1460
1480
1500
1520
1540
1560
1580
1600
1620
1640
1660
1680
1700
1720
1740
1760
1780
1800
1820
1840
1860
1880
1900
1920
1940
1960
1980
2000
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980
1000
1020
1040
1060
1080
1100
1120
1140
1160
1180
1200
1220
1240
1260
1280
1300
1320
1340
1360
1380
1400
1420
1440
1460
1480
1500
1520
1540
1560
1580
1600
1620
1640
1660
1680
1700
1720
1740
1760
1780
1800
1820
1840
1860
1880
1900
1920

In [6]:
tr_data

(<tf.Tensor 'IteratorGetNext:0' shape=(?,) dtype=uint16>,
 <tf.Tensor 'IteratorGetNext:1' shape=(?, 300, 3) dtype=uint8>)

In [23]:
tr = np.concatenate(tr_feats).astype(np.float32)
ev = np.concatenate(ev_feats).astype(np.float32)
np.save(features_dir + '/tr_logits_cnn.npy', tr)
np.save(features_dir + '/ev_logits_cnn.npy', ev)